In [1]:
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
from gaGen import GenGeneticAlgorithm
import copy

In [2]:
def evaluate(X_train : pd.DataFrame, y_train: pd.DataFrame, X_test: pd.DataFrame, y_test: pd.DataFrame, random_state = None):
    
    clf = DecisionTreeClassifier(random_state= random_state)
    
    clf.fit(X_train, y_train)

    predictions = clf.predict(X_test)
    precision = precision_score(predictions,y_test, average='macro')
    f1 = f1_score(predictions,y_test, average='macro')
    recall = recall_score(predictions,y_test, average='macro')
    
    return {"precision":precision, "f1":f1, "recall":recall}

In [3]:
dataset_name= "vehicule"

data = pd.read_csv(f"../Datasets/{dataset_name}.csv")

data.head(10)

,COMPACTNESS,CIRCULARITY,'DISTANCE CIRCULARITY','RADIUS RATIO','PR.AXIS ASPECT RATIO','MAX.LENGTH ASPECT RATIO','SCATTER RATIO',ELONGATEDNESS,'PR.AXIS RECTANGULARITY','MAX.LENGTH RECTANGULARITY','SCALED VARIANCE_MAJOR','SCALED VARIANCE_MINOR','SCALED RADIUS OF GYRATION','SKEWNESS ABOUT_MAJOR','SKEWNESS ABOUT_MINOR','KURTOSIS ABOUT_MAJOR','KURTOSIS ABOUT_MINOR','HOLLOWS RATIO',Class
0,95,48,83,178,72,10,162,42,20,159,176,379,184,70,6,16,187,197,van
1,91,41,84,141,57,9,149,45,19,143,170,330,158,72,9,14,189,199,van
2,104,50,106,209,66,10,207,32,23,158,223,635,220,73,14,9,188,196,saab
3,93,41,82,159,63,9,144,46,19,143,160,309,127,63,6,10,199,207,van
4,85,44,70,205,103,52,149,45,19,144,241,325,188,127,9,11,180,183,bus
5,107,57,106,172,50,6,255,26,28,169,280,957,264,85,5,9,181,183,bus
6,97,43,73,173,65,6,153,42,19,143,176,361,172,66,13,1,200,204,bus
7,90,43,66,157,65,9,137,48,18,146,162,281,164,67,3,3,193,202,van
8,86,34,62,140,61,7,122,54,17,127,141,223,112,64,2,14,200,208,van
9,93,44,98,197,62,11,183,36,22,146,202,505,152,64,4,14,195,204,saab


In [4]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [5]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state= 42)

before_scores= []
selected_scores= []
all_scores = []
columns_number= []

for index,(train_index, test_index) in enumerate(skf.split(X, y)):
    
    fold_x_train =X.iloc[train_index]
    fold_y_train = y.iloc[train_index]
    
    fold_x_test =X.iloc[test_index]
    fold_y_test = y.iloc[test_index]
    
    score_before = evaluate(fold_x_train, fold_y_train, fold_x_test, fold_y_test)
    before_scores.append(score_before)
    
    print(f'''For the original data:
    Columns : {fold_x_train.columns}
    Columns Number : {len(fold_x_train.columns)} 
    precision : {score_before["precision"]}
    f1        : {score_before["f1"]}
    recall    : {score_before["recall"]}
          ''')
    print("Ga Gen started with inputs ......")
    ga= GenGeneticAlgorithm(
        X_train= fold_x_train,
        X_test= fold_x_test,
        y_train= fold_y_train,
        y_test= fold_y_test,
        rnd_state=None
    )

    selected_new_X_train , selected_new_X_test, all__new_X_train ,all_new_X_test = ga.run(population_size= 30, generation_limit= 50, mutation_probability= 0.1)
    
    print("Ga Gen finished ...")

    score_after= evaluate(all__new_X_train, fold_y_train, all_new_X_test, fold_y_test)
    all_scores.append(score_after)
    
    print(f'''\nFor the new generated data with gaGen without selection:
    Columns : {all_new_X_test.columns}
    Columns Number : {len(all_new_X_test.columns)} 
    precision : {score_after["precision"]}
    f1        : {score_after["f1"]}
    recall    : {score_after["recall"]}
    ''')
    score_after= evaluate(selected_new_X_train, fold_y_train, selected_new_X_test, fold_y_test)
    selected_scores.append(score_after)
    
    columns_number.append(len(selected_new_X_test.columns))
    
    print(f'''\nFor the new generated data with gaGen with selection:
    Columns : {selected_new_X_test.columns}
    Columns Number : {len(selected_new_X_test.columns)} 
    precision : {score_after["precision"]}
    f1        : {score_after["f1"]}
    recall    : {score_after["recall"]}
--------------------------------------------------------------------------------------''')
    

For the original data:
    Columns : Index(['COMPACTNESS', 'CIRCULARITY', ''DISTANCE CIRCULARITY'',
       ''RADIUS RATIO'', ''PR.AXIS ASPECT RATIO'', ''MAX.LENGTH ASPECT RATIO'',
       ''SCATTER RATIO'', 'ELONGATEDNESS', ''PR.AXIS RECTANGULARITY'',
       ''MAX.LENGTH RECTANGULARITY'', ''SCALED VARIANCE_MAJOR'',
       ''SCALED VARIANCE_MINOR'', ''SCALED RADIUS OF GYRATION'',
       ''SKEWNESS ABOUT_MAJOR'', ''SKEWNESS ABOUT_MINOR'',
       ''KURTOSIS ABOUT_MAJOR'', ''KURTOSIS ABOUT_MINOR'', ''HOLLOWS RATIO''],
      dtype='object')
    Columns Number : 18 
    precision : 0.7076298701298701
    f1        : 0.7048980862659991
    recall    : 0.7046750398724082
          
Ga Gen started with inputs ......
Ga Gen finished ...

For the new generated data with gaGen without selection:
    Columns : Index(['COMPACTNESS', 'CIRCULARITY', ''DISTANCE CIRCULARITY'',
       ''RADIUS RATIO'', ''PR.AXIS ASPECT RATIO'', ''MAX.LENGTH ASPECT RATIO'',
       ''SCATTER RATIO'', 'ELONGATEDNESS', ''PR.A

In [6]:
sums = {}
counts = {}

for score in before_scores:
    for key, value in score.items():
        sums[key] = sums.get(key, 0) + value
        counts[key] = counts.get(key, 0) + 1

means = {key: sums[key] / counts[key] for key in sums}

print("Original Data:")
for prop, mean in means.items():
    print(prop + ":", mean)

Original Data:
precision: 0.7182404875825928
f1: 0.7186312124382619
recall: 0.7239096364695732


In [7]:
sums = {}
counts = {}

for score in all_scores:
    for key, value in score.items():
        sums[key] = sums.get(key, 0) + value
        counts[key] = counts.get(key, 0) + 1

means = {key: sums[key] / counts[key] for key in sums}

print("Generated without fs Data:")
for prop, mean in means.items():
    print(prop + ":", mean)

Generated without fs Data:
precision: 0.7576660401002505
f1: 0.7558573117000649
recall: 0.7585813206466809


In [8]:
sums = {}
counts = {}

for score in selected_scores:
    for key, value in score.items():
        sums[key] = sums.get(key, 0) + value
        counts[key] = counts.get(key, 0) + 1

means = {key: sums[key] / counts[key] for key in sums}

print("Generated with fs Data:")
for prop, mean in means.items():
    print(prop + ":", mean)

Generated with fs Data:
precision: 0.7314197425381636
f1: 0.733113059799032
recall: 0.7413600639930058


In [9]:
print(f"Columns generation: {np.mean(columns_number)}")

Columns generation: 233.1
